# 1-2


In [4]:
using JuMP, HiGHS


I = 1:4    # stations
J = 1:8    # jobs: 1=(T1,S1),2=(T1,S2),3=(T1,S3),4=(T2,S1),5=(T2,S2),6=(T3,S1),7=(T3,S2),8=(T3,S3)

p = [
    # j =  1   2   3   4   5   6   7   8
    # T1   T1  T1  T2  T2  T3  T3  T3
    17  20  17  13  19  11  13   9;   # i=1
    18  18  21  18  21  13   9  13;   # i=2
    12  17   9  21  17   9   9  17;   # i=3
    16  14  21  12  15  14  12  17;   # i=4
]

# availability a[i] and setup s[i]
a = [41, 45, 50, 46]
s = [10, 12,  8,  6]

model = Model(HiGHS.Optimizer)

@variable(model, z[j in J, i in I], Bin)   # z[j,i] = 1 if job j on station i
@variable(model, x[i in I], Bin)           # x[i]=1 if station i is used
@variable(model, C >= 0)                   # makespan

@constraint(model, [j in J], sum(z[j,i] for i in I) == 1)

@constraint(model, [i in I], z[1,i] == z[2,i])
@constraint(model, [i in I], z[2,i] == z[3,i])

@constraint(model, [i in I],
    s[i]*x[i] + sum(p[i,j]*z[j,i] for j in J) <= a[i]
)

# Link usage
@constraint(model, [j in J, i in I], z[j,i] <= x[i])

# Makespan definition
@constraint(model, [i in I],
    s[i]*x[i] + sum(p[i,j]*z[j,i] for j in J) <= C
)

@objective(model, Min, C)

set_silent(model)
optimize!(model)

status = termination_status(model)
@show status
println("Min time to complete: $(round(value(C); digits=2)) min")

println("\nJob → Station assignments:")
for j in J, i in I
    if value(z[j,i]) > 0
        println("  Job $j → Station $i")
    end
end

println("\nStation workloads (including setup):")
for i in I
    load = s[i]*value(x[i]) + sum(p[i,j]*value(z[j,i]) for j in J)
    println("  Station $i: $(round(load; digits=2)) min ", 
            "(used=$(value(x[i])>0.5))")
end


status = MathOptInterface.OPTIMAL
Min time to complete: 46.0 min

Job → Station assignments:
  Job 1 → Station 3
  Job 2 → Station 3
  Job 3 → Station 3
  Job 4 → Station 4
  Job 5 → Station 4
  Job 6 → Station 1
  Job 7 → Station 4
  Job 8 → Station 1

Station workloads (including setup):
  Station 1: 30.0 min (used=true)
  Station 2: 0.0 min (used=false)
  Station 3: 46.0 min (used=true)
  Station 4: 45.0 min (used=true)


# 3-2


In [7]:
using JuMP, GLPK

I = 1:8 # districts
p = [40, 30, 35, 20, 15, 50, 45, 60]                # populations (thousands)
d = [
    0  3  4  6  1  9  8 10;
    3  0  5  4  8  6  1  9;
    4  5  0  2  2  3  5  7;
    6  4  2  0  3  2  5  4;
    1  8  2  3  0  2  2  4;
    9  6  3  2  2  0  3  2;
    8  1  5  5  2  3  0  2;
   10  9  7  4  4  2  2  0
]
n = [d[i,j] ≤ 2 ? 1 : 0 for i in I, j in I]         # reachable within 2 s

# Model
model = Model(GLPK.Optimizer)
@variable(model, a[i in I], Bin)                   # auror at i?
@variable(model, c[i in I], Bin)                   # i covered?

@constraint(model, sum(a[j] for j in I) == 3)      # exactly 3 aurors
@constraint(model, [i in I],
    c[i] <= sum(n[i,j] * a[j] for j in I)           # coverage if ≤2 s away
)

@objective(model, Max, sum(p[i] * c[i] for i in I)) # maximize covered population

optimize!(model)

# Results
println("Auror locations: ", [j for j in I if value(a[j]) > 0])
println("Covered districts: ", [i for i in I if value(c[i]) > 0])
println("Total covered population (thousands): ", objective_value(model))


Auror locations: [1, 4, 7]
Covered districts: [1, 2, 3, 4, 5, 6, 7, 8]
Total covered population (thousands): 295.0


# 4-3

In [11]:
using JuMP, GLPK, NamedArrays

clean_duration_matrix = [
     0  5  9 20 14 14 10  8;
    19  0 10 20  9  6 13  6;
    12 13  0 18 19 20 20 11;
    20 21 22  0 25 19 20 25;
     7 11 18  9  0  9  9 15;
    12 10 16  5 16  0  9  7;
    14 16 11  5 14 15  0 11;
    10 11  7 14  8 19 16  0
]

job_pairs = [
    (:1,:1), (:1,:2), (:1,:3),
    (:2,:1), (:2,:2),
    (:3,:1), (:3,:2), (:3,:3)
]

duration_NA = NamedArray(clean_duration_matrix,
                  (job_pairs, job_pairs), ("job pair","job pair"))

make_times = Dict(zip(job_pairs,[10, 13, 7, 12, 11, 8, 8, 9]))

clean = Matrix(duration_NA)
make  = [make_times[j] for j in job_pairs]
names = [string(tp[1], "T", tp[2], "S") for tp in job_pairs]  # nicer labels

n = length(names)

m = Model(GLPK.Optimizer)

@variable(m, x[1:n,1:n], Bin)
@variable(m, u[1:n])

# exactly one next_jobessor / predecessor for every job
for i in 1:n
    @constraint(m, x[i,i] == 0)
    @constraint(m, sum(x[i,j] for j in 1:n) == 1)
    @constraint(m, sum(x[j,i] for j in 1:n) == 1)
end

# MTZ (root at node 1 only to break symmetry)
@constraint(m, u[1] == 0)
for i in 2:n
    @constraint(m, 1 <= u[i] <= n-1)
end
for i in 2:n, j in 2:n
    if i != j
        @constraint(m, u[i] - u[j] + (n-1)*x[i,j] ≤ n-2)
    end
end

@objective(m, Min, sum(clean[i,j]*x[i,j] for i=1:n, j=1:n) + sum(make))

optimize!(m)

# recover the tour 
next_job = Dict(i => findfirst(j -> value(x[i,j]) > 0, 1:n) for i in 1:n)
cycle = Vector{Int}()
cur = 1
for _ in 1:n
    push!(cycle, cur)
    cur = next_job[cur]
end

# rotate so the wrap-around cleaning arc is the shortest one
wrap_costs = [clean[cycle[end - i], cycle[1 + i]] for i in 0:n-1]
rot = findmin(wrap_costs)[2] - 1           # find the min edge from the previous day to the first job
println("Rotation: $rot")
cycle = circshift(cycle, -rot)

clean_time = sum(clean[i,j]*value(x[i,j]) for i=1:n, j=1:n)
total_time = clean_time + sum(make)

println("Optimal cyclic order (any rotation is equivalent):")
println(join(names[cycle], " → "))

println("\nDaily minutes")
println("  making time : ", sum(make))
println("  cleaning time : ", clean_time)
println("  TOTAL      : ", total_time)



Rotation: 4
Optimal cyclic order (any rotation is equivalent):
2T1S → 3T1S → 3T3S → 1T3S → 1T1S → 1T2S → 2T2S → 3T2S

Daily minutes
  making time : 78
  cleaning  time : 73.0
  TOTAL      : 151.0


# 5-2

In [17]:
# ───────────  Uber relocation – multiple-choice formulation  ───────────
using JuMP, GLPK

loc  = ["CapitolSq","Epic","Overture","CampRand","Hilldale",
        "WestTowne","Olbrich","Arboretum","PicnicPt","Garver"]
coord = [(0,0),(20,20),(18,10),(30,12),(35,0),
         (33,25),(4,27),(4,10),(11,0),(2,15)]
supply  = [6,11,4,8,12,2,14,11,15,7]
demand = [8,6,8,9,9,7,13,7,9,14]
N = length(loc)

# Euclidean distances (miles)
dist = [sqrt((coord[i][1]-coord[j][1])^2 + (coord[i][2]-coord[j][2])^2)
        for i in 1:N, j in 1:N]

# breakpoints and piecewise parameters
B = [0,2,5,8]                       # 0-2, 2-5, 5-8 cars
Δ = diff(B)                         # [2,3,3]
c = [5,3,1]                         # $/mile, slopes
p = [0,4,14]                        # intercepts

b = supply .- demand             

mip = Model(GLPK.Optimizer)

@variable(mip, 0 <= w[1:N,1:N,1:3] <= B[end])   # flow if tier chosen
@variable(mip, z[1:N,1:N,1:3], Bin)             # tier-choice

#   B_{s-1} z ≤ w ≤ B_s z
for i in 1:N, j in 1:N, s in 1:3
    @constraint(mip, w[i,j,s] >= B[s]*z[i,j,s])      # lower
    @constraint(mip, w[i,j,s] <= B[s+1]*z[i,j,s])    # upper
end
# exactly one piece active 
@constraint(mip, [i=1:N, j=1:N], sum(z[i,j,s] for s=1:3) == 1)

# meet demand at every location
for i in 1:N
    @constraint(mip,
        sum(w[i,j,s] for j=1:N, s=1:3) -
        sum(w[j,i,s] for j=1:N, s=1:3) == b[i])
end
# Actually no need to create another variable nij to represent movement, wij is enough
@objective(mip, Min,
    sum( (c[s]*w[i,j,s] + p[s]*z[i,j,s]) * dist[i,j]
         for i=1:N, j=1:N, s=1:3) )

optimize!(mip)

# ── display results ───────────────────────────────────────────────────
println("\nCar movements:")
for i in 1:N, j in 1:N
    moved = sum(value.(w[i,j,s]) for s=1:3)
    if moved > 1e-6 && i != j
        println(rpad(loc[i],14)," → ",lpad(loc[j],14),": ",
                round(moved), " car(s)")
    end
end
println("\nTotal cost \$", objective_value(mip))




Car movements:
Epic           →      WestTowne: 5.0 car(s)
Hilldale       →       Overture: 2.0 car(s)
Hilldale       →       CampRand: 1.0 car(s)
Olbrich        →         Garver: 1.0 car(s)
Arboretum      →         Garver: 6.0 car(s)
PicnicPt       →      CapitolSq: 2.0 car(s)
PicnicPt       →       Overture: 2.0 car(s)
PicnicPt       →      Arboretum: 2.0 car(s)

Total cost $1049.5322402600048


# 6-1

In [38]:
using JuMP, HiGHS   # HiGHS = fast open-source MILP solver

houses = 1:5

colours      = [:red,:green,:white,:yellow,:blue]
nations      = [:Brit,:Swede,:Dane,:Norwegian,:German]
drinks       = [:tea,:coffee,:milk,:beer,:water]
cigars       = [:PallMall,:Dunhill,:Blends,:BlueMaster,:Prince]
pets         = [:dog,:bird,:cat,:horse,:fish]

attributes = Dict(:colour=>colours, :nation=>nations, :drink=>drinks,
            :smoke=>cigars, :pet=>pets)

m = Model(HiGHS.Optimizer)

@variable(m, x[a in keys(attributes), h in houses, v in attributes[a]], Bin)

# Each house has only one attribute of each type
for a in keys(attributes), h in houses
    @constraint(m, sum(x[a,h,v] for v in attributes[a]) == 1)
end
for a in keys(attributes), v in attributes[a]
    @constraint(m, sum(x[a,h,v] for h in houses) == 1)
end

# 1. The Brit live in red house
@constraint(m, [h in houses], x[:nation,h,:Brit] - x[:colour,h,:red] == 0)
# 2. Swede owns dogs
@constraint(m, [h in houses], x[:nation,h,:Swede] - x[:pet,h,:dog] == 0)
# 3. Dane drinks tea
@constraint(m, [h in houses], x[:nation,h,:Dane] - x[:drink,h,:tea] == 0)
# 4. green is one position to the left of white
@constraint(m, [h in 1:4], x[:colour,h,:green] - x[:colour,h+1,:white] == 0)
# 5. green house owner drinks coffee
@constraint(m, [h in houses], x[:colour,h,:green] - x[:drink,h,:coffee] == 0)
# 6. PallMall = birds
@constraint(m, [h in houses], x[:smoke,h,:PallMall] - x[:pet,h,:bird] == 0)
# 7. yellow = Dunhill
@constraint(m, [h in houses], x[:colour,h,:yellow] - x[:smoke,h,:Dunhill] == 0)
# 8. centre house (h=3) drinks milk
@constraint(m, x[:drink,3,:milk] == 1)
# 9. Norwegian in first house
@constraint(m, x[:nation,1,:Norwegian] == 1)

#10. Blends next to cats
for h in houses
    left  = h > 1  ? x[:pet,h-1,:cat] : 0
    right = h < 5 ? x[:pet,h+1,:cat] : 0
    @constraint(m, x[:smoke,h,:Blends] <= left + right) # if the house has blends, either left or right has cats
end
#11 horses next to Dunhill
for h in houses
    left  = h > 1  ? x[:smoke,h-1,:Dunhill] : 0
    right = h < 5 ? x[:smoke,h+1,:Dunhill] : 0
    @constraint(m, x[:pet,h,:horse] <= left + right)
end
#12 BlueMaster = beer
@constraint(m, [h in houses], x[:smoke,h,:BlueMaster] - x[:drink,h,:beer] == 0)
#13 German = Prince
@constraint(m, [h in houses], x[:nation,h,:German] - x[:smoke,h,:Prince] == 0)
#14 Norwegian next to blue
@constraint(m, x[:nation,1,:Norwegian] <= x[:colour,2,:blue])
for h in 2:4
    @constraint(m, x[:nation,h,:Norwegian] <= x[:colour,h-1,:blue] + x[:colour,h+1,:blue])
end
@constraint(m, x[:nation,5,:Norwegian] <= x[:colour,4,:blue])
#15 Blends neighbour drinks water
for h in houses
    left  = h > 1  ? x[:drink,h-1,:water] : 0
    right = h < 5 ? x[:drink,h+1,:water] : 0
    @constraint(m, x[:smoke,h,:Blends] <= left + right)
end
set_silent(m)
optimize!(m)


# print a table of the houses and attributes
for a in keys(attributes)
    # tabular
    print("\t$(a)")
end

for h in houses
    println()
    print("House $h")
    for a in keys(attributes)
        print("\t$(first(v for v in attributes[a] if value(x[a,h,v]) > 0))")
    end
end

println()
fish_house = first(h for h in houses if value(x[:pet,h,:fish]) > 0.5)
owner      = first(n for n in nations if value(x[:nation,fish_house,n]) > 0.5)
println("Fish is kept in house #$fish_house by the $owner.")


	drink	colour	pet	nation	smoke
House 1	water	yellow	cat	Norwegian	Dunhill
House 2	tea	blue	horse	Dane	Blends
House 3	milk	red	bird	Brit	PallMall
House 4	coffee	green	fish	German	Prince
House 5	beer	white	dog	Swede	BlueMaster
Fish is kept in house #4 by the German.
